In [ ]:
!gdown --id 1jcd9ocuh50lnwrDSiY10FbYu5X6mDpCJ

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1jcd9ocuh50lnwrDSiY10FbYu5X6mDpCJ
To: /content/extracted_triples_2.json
100% 97.2k/97.2k [00:00<00:00, 6.22MB/s]


In [ ]:
!python -m spacy download en_core_web_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 15.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
!pip install pandas

In [ ]:
import json
import pandas as pd
import spacy

In [ ]:
nlp =  spacy.load('en_core_web_md')

# Векторные представления

In [ ]:
nlp_similarity = {
    'COUNTRY': {"baseline":'country', 'flag': True},
    'AWARD_RECEIVED':{"baseline":'award', 'flag': True},
    'PLATFORM':{"baseline":'platform', 'flag': True},
    'LEAGUE': {"baseline":'league', 'flag': True},
    'LOCATED_IN_OR_NEXT_TO_BODY_OF_WATER': {"baseline":'water', 'flag': True},
    'BASIN_COUNTRY': {"baseline":'water', 'flag': False},
    'CONTINENT': {"baseline":'continent', 'flag': True},
    'RECORD_LABEL': {"baseline":'record company', 'flag': True}
}

def similarity_check_nlp(triple):
    is_corrected = False
    subj = nlp(triple['subj'][0][0])
    obj = nlp(triple['obj'][0][0])
    pred = nlp(nlp_similarity[triple['pred']]['baseline'])
    flag = nlp_similarity[triple['pred']]['flag']
    p_s = pred.similarity(subj)
    p_o = pred.similarity(obj)
    if flag:
        if p_s > p_o:
            correct_triple = {'subj': triple['obj'],'obj':triple['subj'],"pred": triple['pred']}
            is_corrected = True
    else:
        if p_s < p_o:
            correct_triple = {'subj': triple['obj'],'obj':triple['subj'],"pred": triple['pred']}
            is_corrected = True
    return (correct_triple if is_corrected else triple,is_corrected)

#Правила

In [ ]:
correct_types = {
    'LOCATED_IN_THE_ADMINISTRATIVE_TERRITORIAL_ENTITY': {"subj": ['LOC','ORG','MISC'], 'obj': ['LOC','ORG','MISC'],'correct':{'subj': ['ORG','MISC','LOC'],'obj':'LOC'}},
    'INCEPTION': {'subj': ['ORG','MISC','TIME','LOC'],'obj':['ORG','MISC','TIME','LOC'], 'correct':{'subj': ['ORG','MISC','LOC'],'obj':'TIME'}},
    'LANGUAGES_SPOKEN_WRITTEN_OR_SIGNED': {"subj": ['PER','MISC','LOC'], 'obj': ['PER','MISC', 'LOC'],'correct':{'subj': 'PER','obj':['MISC','LOC']}},
    'COUNTRY': {"subj": ['LOC','MISC','ORG','PER'],'obj':['LOC','MISC','ORG','PER'], 'correct':{'subj': ['LOC','MISC','ORG','PER'],'obj':'LOC'}},
    'CHAIRPERSON': {"subj": ['PER','ORG'], 'obj': ['PER','ORG'], 'correct':{'subj': 'ORG','obj':'PER'}},
    'MEMBER_OF_POLITICAL_PARTY': {"subj": ['PER','ORG'], 'obj': ['PER','ORG'], 'correct':{'subj': 'PER','obj':'ORG'}},
    'HEAD_OF_GOVERNMENT': {"subj": ['PER','LOC','ORG'], 'obj': ['PER','LOC','ORG'], 'correct':{'subj':'PER' ,'obj':['LOC','ORG']}},
    'DATE_OF_BIRTH': {"subj": ['PER','TIME'], 'obj': ['PER','TIME'],'correct':{'subj':'PER' ,'obj':'TIME'}},
    'DATE_OF_DEATH': {"subj": ['PER','TIME'], 'obj': ['PER','TIME'],'correct':{'subj':'PER' ,'obj':'TIME'}},
    'SIBLING': {"subj": ['PER'], 'obj': ['PER'],'correct':{'subj':'PER' ,'obj':'PER'}, 'reverse': 'SIBLING'},
    'PLACE_OF_BIRTH': {'subj': ['LOC','PER''ORG'],'obj':['LOC','PER','ORG'],'correct':{'subj': 'PER','obj':['LOC','ORG']}},
    'PLACE_OF_DEATH': {'subj': ['LOC','PER''ORG'],'obj':['LOC','PER''ORG'],'correct':{'subj': 'PER','obj':['LOC','ORG']}},
    'COUNTRY_OF_CITIZENSHIP': {"subj": ["PER", "LOC", "ORG", "MISC"], 'obj': ["PER", "LOC", "ORG", "MISC"], 'correct':{'subj': ["PER","ORG", "MISC"],'obj':'LOC'}},
    'POSITION_HELD': {"subj": ['PER','MISC','ORG'], 'obj': ['PER','MISC','ORG'],'correct':{'subj': 'PER','obj': ['ORG','MISC']}},
    'PARTICIPANT': {'subj': ['PER','ORG','LOC','MISC'],'obj': ['PER','ORG','LOC','MISC'], 'correct':{'subj': 'MISC','obj':['PER','ORG','LOC']}, 'reverse': 'PARTICIPANT_OF'},
    'AUTHOR': {"subj": ['MISC','PER','ORG'], 'obj': ['MISC','PER','ORG'], 'correct':{'subj': 'MISC','obj':['PER','ORG']}},
    'DISSOLVED_ABOLISHED_OR_DEMOLISHED': {"subj": ['TIME','ORG','LOC','MISC'], 'obj': ['TIME','ORG','LOC','MISC'],'correct':{'subj': ['ORG','LOC','MISC'],'obj': 'TIME'}},
    'CONTAINS_ADMINISTRATIVE_TERRITORIAL_ENTITY' : {"subj": ['LOC','ORG'], 'obj': ['LOC','ORG'], 'correct':{'subj': 'LOC','obj':['LOC','ORG']}},
    'HEADQUARTERS_LOCATION': {"subj": ['LOC','ORG'], 'obj': ['LOC','ORG'], 'correct':{'subj': 'ORG','obj':'LOC'}},
    'PUBLICATION_DATE': {"subj": ['TIME','MISC'], 'obj': ['TIME','MISC'],'correct':{'subj': 'MISC','obj':'TIME'}},
    'NOTABLE_WORK': {"subj": ['PER','MISC','ORG'], 'obj': ['PER','MISC','ORG'], 'correct':{'subj': ['PER','ORG'],'obj':'MISC'}},
    'AWARD_RECEIVED': {'subj':['MISC','PER','ORG'],'obj':['MISC','PER','ORG'], 'correct':{'subj': ['PER','MISC','ORG'],'obj':'MISC'}},
    'OWNED_BY': {"subj": ['MISC','ORG','PER','LOC'], 'obj': ['MISC','ORG','PER','LOC'], 'correct':{'subj': ['MISC','ORG','LOC'],'obj': ['PER','ORG']}},
    'CAST_MEMBER': {"subj": ['MISC','PER'], 'obj': ['MISC','PER'],'correct':{'subj': 'MISC','obj':'PER'}},
    'TERRITORY_CLAIMED_BY': {"subj": ['LOC'], 'obj': ['LOC'], 'correct':{'subj': 'LOC','obj':'LOC'}},
    'PERFORMER': {"subj": ['PER','MISC'], 'obj': ['PER','MISC'], 'correct':{'subj': 'PER','obj':'MISC'}},
    'LOCATION_OF_FORMATION': {"subj": ['ORG','LOC'], 'obj': ['ORG','LOC'],'correct':{'subj': 'ORG','obj':'LOC'}},
    'COUNTRY_OF_ORIGIN': {"subj": ['LOC','ORG','MISC','PER'], 'obj': ['LOC','ORG','MISC','PER'], 'correct':{'subj': ['ORG','MISC','PER'],'obj':'LOC'}},
    'FATHER': {"subj": ['PER'], 'obj': ['PER'], 'correct':{'subj': 'PER','obj':'PER'}, 'reverse': 'CHILD'},
    'MOTHER': {"subj": ['PER'], 'obj': ['PER'], 'correct':{'subj': 'PER','obj':'PER'}, 'reverse': 'CHILD'},
    'HEAD_OF_STATE': {'subj': ['LOC','PER'],'obj':['LOC','PER'], 'correct':{'subj': 'LOC','obj':'PER'}},
    'CHILD': {"subj": ['PER'], 'obj': ['PER'], 'correct':{'subj': 'PER','obj':'PER'}},
    'SPOUSE': {"subj": ['PER'], 'obj': ['PER'], 'correct':{'subj': 'PER','obj':'PER'}, 'reverse': 'SPOUSE'},
    'START_TIME': {"subj": ['TIME','MISC'], 'obj': ['TIME','MISC'],'correct':{'subj': 'MISC','obj':'TIME'}},
    'END_TIME': {'subj': ['TIME','MISC'],'obj':['TIME','MISC'],'correct':{'subj': 'MISC','obj':'TIME'}},
    'POINT_IN_TIME': {'subj': ['TIME','MISC'],'obj':['TIME','MISC'],'correct':{'subj': 'MISC','obj':'TIME'}},
    'ORIGINAL_NETWORK': {"subj": ['MISC',"ORG"], 'obj': ['MISC','ORG'],'correct':{'subj': 'MISC','obj':'ORG'}},
    'HAS_PART': {"subj": ['LOC','ORG','MISC'], 'obj': ['LOC','ORG','MISC'], 'correct':{'subj': ['LOC','ORG','MISC'],'obj':['LOC','ORG','MISC']}, 'reverse': "PART_OF"},
    'DIRECTOR': {"subj": ['PER','MISC'], 'obj': ['PER','MISC'], 'correct': {'subj':'MISC','obj':'PER'}},
    'SCREENWRITER': {"subj": ['PER','MISC'], 'obj': ['PER','MISC'], 'correct': {'subj':'MISC','obj':'PER'}},
    'CHARACTERS': {"subj": ['PER','MISC'], 'obj': ['PER','MISC'], 'correct': {'subj':'MISC','obj':'PER'}, 'reverse': 'PRESENT_IN_WORK'},
    'PRESENT_IN_WORK': {'subj': ['PER','MISC'],'obj':['PER','MISC'], 'correct': {'subj':'PER','obj':'MISC'}, 'reverse': 'CHARACTERS'},
    'CREATOR': {"subj": ['PER','MISC'], 'obj': ['PER','MISC'], 'correct': {'subj':'MISC','obj':'PER'}},
    'EMPLOYER': {"subj": ['PER','ORG'], 'obj': ['PER','ORG'], 'correct':{'subj': 'PER','obj':['PER','ORG']}},
    'PRODUCER': {"subj": ['PER','MISC'], 'obj': ['PER','MISC'],'correct': {'subj':'MISC','obj':'PER'}},
    'ORIGINAL_LANGUAGE_OF_WORK': {"subj": ['MISC'], 'obj': ['MISC'], 'correct': {'subj':'MISC','obj':'MISC'}},
    'DEVELOPER': {"subj": ['ORG','MISC','PER'], 'obj': ['ORG','MISC','PER'],'correct': {'subj':'MISC','obj':['PER','ORG']}},
    'PUBLISHER': {"subj": ['ORG','MISC','PER'], 'obj': ['ORG','MISC','PER'],'correct': {'subj':'MISC','obj':['PER','ORG']}},
    'PLATFORM': {"subj": ['MISC'], 'obj': ['MISC'], 'correct': {'subj':'MISC','obj':'MISC'}},
    'PRODUCTION_COMPANY': {"subj": ['ORG','MISC'], 'obj': ['ORG','MISC'],'correct': {'subj':'MISC','obj':'ORG'}},
    'EDUCATED_AT': {"subj": ['PER','ORG','LOC'], 'obj': ['PER','ORG','LOC'], 'correct': {'subj':'PER','obj':['ORG','LOC']}},
    'OPERATOR': {"subj": ['PER','ORG','MISC'], 'obj': ['PER','ORG','MISC'], 'correct': {'subj':'MISC','obj':['ORG','PER']}},
    'LEAGUE': {"subj": ['PER','ORG','MISC'], 'obj': ['PER','ORG','MISC'], 'correct':{'subj': ['PER','ORG'],'obj':['ORG','MISC']}},
    'LOCATED_IN_OR_NEXT_TO_BODY_OF_WATER': {"subj": ['LOC','ORG'], 'obj': ['LOC','ORG'], 'correct':{'subj': ["LOC",'ORG'],'obj':'LOC'}},
    'LOCATION': {"subj": ['LOC','MISC','ORG'], 'obj': ['LOC','MISC','ORG'], 'correct':{'subj': ['MISC','LOC','ORG'],'obj':'LOC'}},
    'LOCATED_ON_TERRAIN_FEATURE': {"subj": ['LOC','MISC','ORG'], 'obj': ['LOC','MISC','ORG'], 'correct':{'subj': ['MISC','LOC','ORG'],'obj':'LOC'}},
    'RELIGION': {"subj": ['ORG','LOC','PER','MISC'], 'obj': ['ORG','LOC','PER','MISC'], 'correct':{'subj': ['PER','LOC','ORG'],'obj':'MISC'}},
    'LYRICS_BY': {"subj": ['PER','MISC'], 'obj': ['PER','MISC'], 'correct':{'subj': 'MISC','obj':'PER'}},
    'RECORD_LABEL': {"subj": ['ORG','MISC','PER'], 'obj': ['ORG','MISC','PER'], 'correct':{'subj': ['MISC','PER','ORG'],'obj':'ORG'}},
    'MILITARY_BRANCH': {"subj": ['PER','ORG','MISC','LOC'], 'obj': ['PER','ORG','MISC','LOC'], 'correct':{'subj': ['PER','ORG','MISC','LOC'],'obj':'ORG'}},
    'CONFLICT': {'subj': ['MISC','PER'],'obj':['MISC','PER'],'correct':{'subj': ['MISC','PER'],'obj':'MISC'}},
    'MEMBER_OF_SPORTS_TEAM': {"subj": ['PER','ORG'], 'obj': ['PER','ORG'], 'correct':{'subj': 'PER','obj':'ORG'}},
    'LEGISLATIVE_BODY': {"subj": ['LOC','ORG','MISC'], 'obj': ['LOC','ORG','MISC'], 'correct':{'subj': ['LOC','ORG','MISC'],'obj':'ORG'}}, # ??????,
    'WORK_LOCATION': {'subj': ['LOC','PER'],'obj':['LOC','PER'], 'correct':{'subj': 'PER','obj':'LOC'}},
    'APPLIES_TO_JURISDICTION': {'subj': ['LOC','ORG','MISC'],'obj':['LOC','ORG','MISC'], 'correct':{'subj': ['MISC','ORG'],'obj':'LOC'}},
    'PRODUCT_OR_MATERIAL_PRODUCED': {"subj": ['ORG','MISC'], 'obj': ['ORG','MISC'], 'correct': {'subj':'ORG','obj':'MISC'}},
    'GENRE': {"subj": ['MISC','PER'], 'obj': ['MISC','PER'], 'correct': {'subj':'MISC','obj':'MISC'}},
    'COMPOSER': {"subj": ['PER','MISC'], 'obj': ['PER','MISC'], 'correct': {'subj':'MISC','obj':'PER'}},
    'MANUFACTURER': {"subj": ['MISC','ORG','PER'], 'obj': ['MISC','ORG','PER'],'correct': {'subj':'MISC','obj':['PER','ORG']}},
    'PARTICIPANT_OF': {"subj": ['PER','MISC','ORG','LOC'], 'obj': ['PER','MISC','ORG','LOC'], 'correct': {'subj':['PER','ORG','LOC'],'obj':'MISC'}, 'reverse': 'PARTICIPANT'},
    'FOUNDED_BY': {"subj": ['PER','MISC','ORG'], 'obj': ['PER','MISC','ORG'],'correct': {'subj':['MISC','ORG'],'obj':'PER'}},
    'SUBSIDIARY': {"subj": ['ORG'], 'obj': ['ORG'], 'correct':{'subj': 'ORG','obj':'ORG'}, 'reverse': 'PARENT_ORGANIZATION'},
    'CAPITAL' : {"subj": ['LOC'], 'obj': ['LOC'], 'correct':{'subj': 'LOC','obj':'LOC'}, 'reverse': 'CAPITAL_OF'},
    'CAPITAL_OF': {"subj": ['LOC'], 'obj': ['LOC'], 'correct':{'subj': 'LOC','obj':'LOC'}, 'reverse': 'CAPITAL'},
    'PARENT_ORGANIZATION': {"subj": ['ORG'], 'obj': ['ORG'], 'correct':{'subj': 'ORG','obj':'ORG'}, 'reverse': 'SUBSIDIARY'},
    'ETHNIC_GROUP': {"subj": ['LOC','ORG','PER','MISC'], 'obj': ['LOC','ORG','PER','MISC'], 'correct':{'subj': ['LOC','ORG','PER'],'obj':['LOC','ORG','MISC']}},
    'BASIN_COUNTRY': {"subj": ['LOC'], 'obj': ['LOC'], 'correct':{'subj': "LOC",'obj':'LOC'}},
    'OFFICIAL_LANGUAGE': {"subj": ['LOC','MISC','ORG'], 'obj': ['LOC','MISC','ORG'], 'correct':{'subj': ["LOC",'ORG'],'obj': 'MISC'}},
    'MOUTH_OF_THE_WATERCOURSE': {"subj": ['LOC'], 'obj': ['LOC'], 'correct':{'subj': "LOC",'obj':'LOC'}},
    'NARRATIVE_LOCATION': {'subj': ['LOC','MISC'],'obj':['LOC','MISC'],'correct':{'subj': 'MISC','obj':'LOC'}},
    'INFLUENCED_BY': {"subj": ['PER','MISC','ORG'], 'obj': ['PER','MISC','ORG'], 'correct':{'subj': ["PER",'ORG','MISC'],'obj': ["PER",'ORG','MISC']}},
    'RESIDENCE': {'subj': ['LOC','PER'],'obj':['LOC','PER'],'correct':{'subj': 'PER','obj':'LOC'}},
    'UNEMPLOYMENT_RATE': {"subj": ["LOC","TIME", "NUM"], "obj":  ["LOC","TIME","NUM"], 'correct':{'subj': 'LOC','obj':['TIME','NUM']}},
    'SISTER_CITY': {"subj": ['LOC'], 'obj': ['LOC'], 'correct':{'subj': 'LOC','obj':'LOC'}, 'reverse': 'SISTER_CITY'},
    'SEPARATED_FROM': {"subj": ['LOC','ORG'], 'obj': ['LOC','ORG'], 'correct':{'subj': ['LOC','ORG'],'obj':['LOC','ORG']}},
    'PARENT_TAXON': {"subj": ['LOC','MISC'], 'obj': ['LOC','MISC'], 'correct':{'subj': ['LOC','MISC'],'obj':['LOC','MISC']}},
    'REPLACES': {"subj": ['LOC','MISC','ORG'], 'obj': ['LOC','MISC','ORG'], 'correct':{'subj': ['LOC','MISC','ORG'],'obj':['LOC','MISC','ORG']}, 'reverse': 'REPLACED_BY'},
    'REPLACED_BY': {"subj": ['LOC','MISC','ORG'], 'obj': ['LOC','MISC','ORG'], 'correct':{'subj': ['LOC','MISC','ORG'],'obj':['LOC','MISC','ORG']}, 'reverse': 'REPLACES'},
    'FOLLOWS': {"subj": ["MISC", "LOC", "ORG", "TIME", "PER"], 'obj': ["MISC", "LOC", "ORG", "TIME", "PER"], 'correct':{'subj': ["MISC", "LOC", "ORG", "TIME", "PER"],'obj':["MISC", "LOC", "ORG", "TIME", "PER"]}, 'reverse': 'FOLLOWED_BY'},
    'FOLLOWED_BY': {"subj": ["MISC", "LOC", "ORG", "TIME", "PER"], 'obj': ["MISC", "LOC", "ORG", "TIME", "PER"], 'correct':{'subj': ["MISC", "LOC", "ORG", "TIME", "PER"],'obj':["MISC", "LOC", "ORG", "TIME", "PER"]},'reverse': 'FOLLOWS'},
    'INSTANCE_OF': {"subj": ['MISC','LOC','ORG','PER'], 'obj': ['MISC','LOC','ORG','PER'], 'correct':{'subj': ['MISC','LOC','ORG','PER'],'obj':['MISC','LOC','ORG','PER']}},  #check
    'CONTINENT': {"subj": ['LOC','ORG','MISC'], 'obj': ['LOC','ORG','MISC'],'correct':{'subj': ['LOC','ORG','MISC'],'obj':'LOC'}},
    'MEMBER_OF': {"subj": ['PER','ORG','MISC'], 'obj': ['PER','ORG','MISC'],'correct':{'subj': 'PER','obj':['ORG','MISC']}},
    'PART_OF':{"subj": ['MISC','ORG','LOC'], 'obj': ['MISC','ORG','LOC'], 'correct':{'subj': ['MISC','ORG','LOC'],'obj':['ORG','MISC']}, 'reverse': "HAS_PART"},
}


unknown_types = {                                     #?
    'SERIES': {"subj": ['?'], 'obj': ['?']},
    'SUBCLASS_OF':{"subj": ['?'], 'obj': ['?']},
}

In [ ]:
def filter_triples_by_type(triples):
    correct = []
    incorrect = []
    for triple in triples:
        if triple['subj'][0][0] == triple['obj'][0][0]:
            incorrect.append(triple)
            continue
        if triple['pred'] not in correct_types.keys():
            incorrect.append(triple)
            continue
        types = correct_types[triple['pred']]
        if triple['subj'][1] in types['subj'] and triple['obj'][1] in types['obj']:
            correct.append(triple)
        else:
            incorrect.append(triple)
    print(f'Correct: {len(correct)} and Incorrect: {len(incorrect)}')
    return correct

In [ ]:
def filter_triples_by_correct_type(triples):
    correct_triples = []
    updated_triples = []
    incorrect_triples = []
    unruled_triples = []
    reverse_triples = []

    correct_triple = None
    is_corrected = False

    for triple in triples:
        if triple['pred'] not in correct_types.keys():
            unruled_triples.append(triple)
            continue
        types = correct_types[triple['pred']]
        correct_subj_types = types['correct']['subj']
        correct_obj_types = types['correct']['obj']
        # Оба типа не списки
        if not isinstance(correct_subj_types,list)and not isinstance(correct_obj_types,list):
            # Правильные типы не равны
            if correct_subj_types!=correct_obj_types:
                if triple['subj'][1] == correct_subj_types and triple['obj'][1] == correct_obj_types:
                    correct_triples.append(triple)
                    continue
                elif triple['subj'][1] == correct_obj_types and triple['obj'][1] == correct_subj_types:
                    correct_triple = {'subj': triple['obj'],'obj':triple['subj'],"pred": triple['pred']}
                    updated_triples.append(correct_triple)
                    is_corrected = True
                    continue
                else:
                    incorrect_triples.append(triple)
                    continue
            # Правильные типы равны
            else:
                if triple['pred'] in nlp_similarity.keys():
                    _triple, is_corrected = similarity_check_nlp(triple)
                    if is_corrected:
                        correct_triple = _triple
                        updated_triples.append(_triple)
                        continue
                    else:
                        correct_triples.append(_triple)
                        continue
                else:
                    #print(triple['pred'])
                    unruled_triples.append(triple)
                    continue
        # Хотя бы один из правильных типов - список
        if (isinstance(correct_subj_types,list)) or isinstance(correct_obj_types,list):
            # Субъект - список, а объект - не список
            if isinstance(correct_subj_types,list) and not isinstance(correct_obj_types,list):
                # Правильный тип объекта есть в списке правильных типов субъекта
                if correct_obj_types in correct_subj_types:
                    set_1_obj = set([correct_obj_types])
                    set_subj = set(correct_subj_types)
                    difference = set_subj.difference(set_1_obj)
                    if triple['subj'][1]!=triple['obj'][1]:
                        if triple['subj'][1] in difference and triple['obj'][1]==correct_obj_types:
                            correct_triples.append(triple)
                        elif triple['obj'][1] in difference and triple['subj'][1]==correct_obj_types:
                            correct_triple = {'subj': triple['obj'],'obj':triple['subj'],"pred": triple['pred']}
                            updated_triples.append(correct_triple)
                            is_corrected = True
                        else:
                            incorrect_triples.append(triple)
                            continue
                    else:
                        if triple['subj'][1] in difference:
                            incorrect_triples.append(triple)
                            continue
                        else:
                            if triple['pred'] in nlp_similarity.keys():
                                _triple, is_corrected = similarity_check_nlp(triple)
                                if is_corrected:
                                    correct_triple = _triple
                                    updated_triples.append(_triple)
                                else:
                                    correct_triples.append(_triple)
                            else:
                                # print(triple['pred'])
                                unruled_triples.append(triple)
                else:
                    if triple['obj'][1] == correct_obj_types:
                        if triple['subj'][1] in correct_subj_types:
                            correct_triples.append(triple)
                        else:
                            incorrect_triples.append(triple)
                            continue
                    elif triple['subj'][1] == correct_obj_types:
                        if triple['obj'][1] in correct_subj_types:
                            correct_triple = {'subj': triple['obj'],'obj':triple['subj'],"pred": triple['pred']}
                            updated_triples.append(correct_triple)
                            is_corrected =True
                        else:
                            incorrect_triples.append(triple)
                            continue
                    else:
                        incorrect_triples.append(triple)
                        continue
            # Субъект - не список, а объект - список
            if not isinstance(correct_subj_types,list) and isinstance(correct_obj_types,list):
                # Правильный тип субъекта есть в списке правильных типов объекта
                if correct_subj_types in correct_obj_types:
                    set_obj = set(correct_obj_types)
                    set_1_subj = set([correct_subj_types])
                    difference = set_obj.difference(set_1_subj)
                    if triple['subj'][1]!=triple['obj'][1]:
                        if triple['obj'][1] in difference and triple['subj'][1]==correct_subj_types:
                            correct_triples.append(triple)
                        elif triple['subj'][1] in difference and triple['obj'][1]==correct_subj_types:
                            correct_triple = {'subj': triple['obj'],'obj':triple['subj'],"pred": triple['pred']}
                            updated_triples.append(correct_triple)
                            is_corrected =True
                        else:
                            incorrect_triples.append(triple)
                            continue
                    else:
                        if triple['subj'][1] in difference:
                            incorrect_triples.append(triple)
                            continue
                        else:
                            if triple['pred'] in nlp_similarity.keys():
                                _triple, is_corrected = similarity_check_nlp(triple)
                                if is_corrected:
                                    correct_triple = _triple
                                    updated_triples.append(_triple)
                                else:
                                    correct_triples.append(_triple)
                            else:
                                # print(triple['pred'])
                                unruled_triples.append(triple)
                else:
                    # Тип субъекта - правильный
                    if triple['subj'][1] == correct_subj_types:
                        # Тип  объекта есть в списке правильных типов объекта
                        if triple['obj'][1] in correct_obj_types:
                            correct_triples.append(triple)
                        # Иначе
                        else:
                            incorrect_triples.append(triple)
                            continue
                    # Тип субъекта не равен правильному типу субъекта, а тип объекта равен правильному типу субъекта -> меняем их местами
                    elif triple['obj'][1] == correct_subj_types:
                        if triple['subj'][1] in correct_obj_types:
                            correct_triple = {'subj': triple['obj'],'obj':triple['subj'],"pred": triple['pred']}
                            updated_triples.append(correct_triple)
                            is_corrected =True
                        else:
                            incorrect_triples.append(triple)
                            continue
                    else:
                        incorrect_triples.append(triple)
                        continue

            if isinstance(correct_subj_types,list) and isinstance(correct_obj_types,list):
                set_subj = set(correct_subj_types)
                set_obj = set(correct_obj_types)
                only_subj = set_subj.difference(set_obj)
                only_obj = set_obj.difference(set_subj)
                both = set_subj.intersection(set_obj)
                if triple['subj'][1] in only_subj and triple['obj'][1] in only_obj:
                    correct_triples.append(triple)
                elif triple['subj'][1] in only_obj and triple['obj'][1] in only_subj:
                    correct_triple = {'subj': triple['obj'],'obj':triple['subj'],"pred": triple['pred']}
                    updated_triples.append(correct_triple)
                    is_corrected =True
                elif triple['subj'][1] in only_subj and triple['obj'][1] in both:
                    correct_triples.append(triple)
                elif triple['subj'][1] in only_obj and triple['obj'][1] in both:
                    correct_triple = {'subj': triple['obj'],'obj':triple['subj'],"pred": triple['pred']}
                    updated_triples.append(correct_triple)
                    is_corrected =True
                elif triple['subj'][1] in both and triple['obj'][1] in only_obj:
                    correct_triples.append(triple)
                elif triple['subj'][1] in both and triple['obj'][1] in only_subj:
                    correct_triple = {'subj': triple['obj'],'obj':triple['subj'],"pred": triple['pred']}
                    updated_triples.append(correct_triple)
                    is_corrected =True
                elif triple['subj'][1] in both and triple['obj'][1] in both:
                    #TODO
                    # print(triple['pred'])
                    if triple['pred'] in nlp_similarity.keys():
                        _triple, is_corrected = similarity_check_nlp(triple)
                        if is_corrected:
                            correct_triple = _triple
                            updated_triples.append(_triple)
                        else:
                            correct_triples.append(_triple)
                    else:
                        unruled_triples.append(triple)
                else:
                    incorrect_triples.append(triple)
                    continue
        if 'reverse' in types.keys():
            if is_corrected:
                new_triple = {'subj': correct_triple['obj'],'obj':correct_triple['subj'],"pred": types['reverse']}
                reverse_triples.append(new_triple)
            else:
                new_triple = {'subj': triple['obj'],'obj':triple['subj'],"pred": types['reverse']}
                reverse_triples.append(new_triple)
        is_corrected = False
        correct_triple = None

    print('Correct: ',len(correct_triples))
    print('Incorrect: ',len(incorrect_triples))
    print('Updated: ',len(updated_triples))
    print('Unruled: ',len(unruled_triples))
    print('Reverse: ',len(reverse_triples))
    combination = correct_triples + updated_triples + unruled_triples + reverse_triples
    print('Combo: ', len(combination))
    # print('\n'.join(list(map(lambda x: str(x),unruled_triples))))
    return combination

In [ ]:
with open ('extracted_triples_2.json') as f:
    triples = json.load(f)

In [ ]:
print(len(triples))
correct_by_overall_type = filter_triples_by_type(triples)
combo = filter_triples_by_correct_type(correct_by_overall_type)
pd.set_option('display.max_rows', None)

949
Correct: 820 and Incorrect: 129


<ipython-input-8-98fed4698998>:19: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  p_o = pred.similarity(obj)


Correct:  316
Incorrect:  44
Updated:  315
Unruled:  145
Reverse:  29
Combo:  805


<ipython-input-8-98fed4698998>:18: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  p_s = pred.similarity(subj)


In [ ]:
df = pd.DataFrame(triples)
df_combo = pd.DataFrame(combo)
print(df_combo.loc[df_combo['pred']=='LEAGUE'])

                                  subj    pred  \
268  [[arsenal arsenal, arsenal], ORG]  LEAGUE   
306            [[calgary flames], ORG]  LEAGUE   
307    [[national hockey league], ORG]  LEAGUE   
595                     [[porto], ORG]  LEAGUE   
596             [[werder bremen], ORG]  LEAGUE   
617                 [[liverpool], ORG]  LEAGUE   

                                 obj  
268          [[premier league], ORG]  
306  [[national hockey league], ORG]  
307   [[twitter wednesday, x], MISC]  
595       [[champions league], MISC]  
596             [[bundesliga], MISC]  
617       [[champions league], MISC]  
